In [ ]:
%run "../keras-retinanet/keras_retinanet/bin/train.py" --weights ../keras-retinanet/Models/resnet50_coco_best_v2.1.0.h5 \
--batch-size 2 --steps 300 --epochs 30 \
--snapshot-path ../Images \
--random-transform \
--config test_config.ini \
csv ../Images/Harbor_Seals_Tile_Old/harbor_train_annotations.csv ../Images/Harbor_Seals_Tile_Old/harbor_classes.csv \
--val-annotations ../Images/Harbor_Seals_Tile_Old/harbor_val_annotations.csv

pre parse
post arg


Using TensorFlow backend.


post generator
Creating model, this may take a second...
model summary here
start training
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30


In [1]:
# code similar to https://github.com/madelinehayes/seabirdNET/blob/master/albatross_detections_FINAL.ipynb and
# the example file in keras-retinanet

In [2]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
from tensorflow import keras

#
import sys
import os

sys.path.insert(0, '../keras-retinanet/keras_retinanet')
  
import models
from utils.image import read_image_bgr
import utils.visualization
import utils.colors
from utils.gpu import setup_gpu

# use this to change which GPU to use
gpu = '1'

# set the modified tf session as backend in keras
setup_gpu(gpu)

# import keras_retinanet
#from keras_retinanet import models
#from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
#from keras_retinanet.utils.visualization import draw_box, draw_caption
#from keras_retinanet.utils.colors import label_color
#from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import time
import json
from random import shuffle

import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session())

In [3]:
#gpu =  ##fix me
#setup_gpu(gpu)

tf.config.threading.set_inter_op_parallelism_threads(12) ##fix me


In [4]:
from utils.image import read_image_bgr, preprocess_image, resize_image
from utils.visualization import draw_box, draw_caption
from utils.colors import label_color

In [5]:
model_path = '../keras-retinanet/Models/resnet50_coco_best_v2.1.0.h5'
model = models.load_model(model_path, backbone_name='resnet50')
labels_to_names = {0: 'seal'}

Using TensorFlow backend.


In [6]:
#model = models.convert_model(model)

In [7]:
image_dir = "../Images/google_earth_test/"

image_list = []
for root, dirs, files in os.walk(image_dir):
    for filename in files:
        if filename.lower().endswith(('.jpg')):
            image_list.append(image_dir + filename)
print(len(image_list))

4


In [8]:
visualize = True
min_score = 0.5

detections = {}

total_time = 0

count = 0
detection_iterations = 10
shuffle(image_list)

for image_path in image_list: 
    if count > detection_iterations:
        break
    else: 
        count +=1
    
    image = read_image_bgr(image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
#try colab ??
print("Finished, time per image:", total_time/len(image_list))

Finished, time per image: 7.163912415504456


In [9]:
print(detections) #empty because model isn't trained

{}


In [10]:
#output: detection boxes (#)/image
#some way to keep track of confidence and use manual review below a certain threshold 
#some way to visualize this or 